In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utils

In [3]:
import tensorflow as tf
import numpy as np
import data
import train
import matplotlib.pyplot as plt
from regularized_explanations import eager_ops, plot
from skimage import feature
import scipy

In [4]:
train.set_up_environment()

In [5]:
model = tf.keras.models.load_model('saved_model.h5')

In [6]:
x_train, y_train, x_test, y_test = data.decoy_mnist()

In [13]:
for c in range(10):
    index = np.where(y_train == c)[0][0]
    
    edges = feature.canny(x_train[index, :, :, 0]).astype(float)
    smoothed_edges = scipy.ndimage.filters.gaussian_filter(edges, 2.0)
    
    eg_saliency = eager_ops.expected_gradients_multi_output(tf.constant(x_train[index:index+1]), 
                                                            x_train, model, 10, k=100)
    eg_saliency = np.abs(eg_saliency[0, c, :, :, 0])
    
    utils.save_image(x_train[index, :, :, 0], 'images/decoy{}.png'.format(c), minval=-0.5, maxval=0.5)
    utils.save_image(eg_saliency, 'images/saliency{}.png'.format(c), minval=0.0, maxval=np.max(eg_saliency))
    utils.save_image(smoothed_edges, 'images/edges{}.png'.format(c), minval=0.0, maxval=np.max(smoothed_edges))